# Proyecto Music Stream

## Fase 2: CONEXIÓN ENTRE PYTHON Y MYSQLWORKBENCH

1. Conexión al servidor de MySQL
2. Creación de las tablas
3. Inserción de los datos en las tablas



## 1. Conexión al servidor

In [13]:
#Importamos las librerias necesarias: mysql.connector y pandas

import mysql.connector
from mysql.connector import errorcode
import pandas as pd 

In [14]:
#Nos conectamos con el servidor de MySQLWorkbench de Adalab utilizando los datos de 'user', 'password' y 'host' 

cnx = mysql.connector.connect(
    user='root', 
    password='AlumnaAdalab',
    host='localhost')

# Creamos el cursor, mycursor y lo definimos:
mycursor = cnx.cursor()

In [15]:
# Creamos el esquema MusicStream dentro de MySQLWorkbench:
mycursor.execute("CREATE DATABASE IF NOT EXISTS MusicStream")

# Confirmamos los cambios realizados usando cnx.commit ():
cnx.commit()

## 2. Creación de las tablas

In [16]:
# Usamos la base de datos MusicStream utilizando mycursor.execute ():

mycursor.execute("USE MusicStream")

In [17]:
#Creamos ambas tablas vacias, tanto Spotify, como LastFM, con sus columnas correspondientes para rellenar.
#Nota: en las columnas 'Track' de tabla Spotify, y 'Playcount'/'Listeners' de la tabla LastFM:
#tuvimos que poner VARCHAR (1000), y BIGINT porque si no, nos daba error que el contenido era demasiado largo. 

mycursor.execute('''CREATE TABLE IF NOT EXISTS Spotify (
                 Artist VARCHAR (500),
                 Genre VARCHAR (500),
                 Album VARCHAR (500),
                 Track VARCHAR (1000),     
                 Year INT,
                 Spotify_id VARCHAR (2000)
                 )
                 
                 ''')

mycursor.execute('''CREATE TABLE IF NOT EXISTS LastFM (
                 Artist VARCHAR (500),
                 Biography TEXT,
                 Playcount BIGINT,
                 Listeners BIGINT,
                 Similars TEXT
                 )
                 
                 ''')

cnx.commit()



In [18]:

# Consultas de inserción 
query_spotify = """
    INSERT INTO Spotify (Artist, Genre, Album, Track, Year, Spotify_id)  
    VALUES (%s, %s, %s, %s, %s, %s)
"""

query_lastfm = """
    INSERT INTO LastFM (Artist, Biography, Playcount, Listeners, Similars)  
    VALUES (%s, %s, %s, %s, %s)
"""

#Con las variables df_spotify y df_dataframe, usando pandas (pd), leemos los ficheros .csv para extraer los datos.
#fill.na se utiliza para la limpieza final de los datos dentro del dataframe. 
#Reemplaza los valores faltantes o nulos (normalmente representados como NaN en Python, y None en SQL). 
#Usando fill.na SQL puede interpretar correctamente los datos que vienen de Python. 
#SQL no reconoce valores faltantes o nulos representados como NaN y por eso los estamos reemplazando con este operador. 

df_spotify = pd.read_csv("spotify_data.csv").fillna("")  
df_lastfm = pd.read_csv("lastfm_data.csv").fillna("")  

#Definimos la variable de spotify_data, convirtiéndola a una lista de listas
#Para hacerlo, se utiliza el operador values.tolist()

spotify_data = df_spotify[['artist', 'genre', 'album', 'track', 'year', 'spotify_id']].values.tolist()
lastfm_data = df_lastfm[['Artist', 'Biography', 'Playcount', 'Listeners', 'Similars']].values.tolist()

# Inserción de datos de las ambas tablas en MySQL Workbench (utilizando executemany)

mycursor.executemany(query_spotify, spotify_data)
mycursor.executemany(query_lastfm, lastfm_data)

#Guardamos los cambios

cnx.commit()
print("Datos insertados correctamente.")

mycursor.close()  # Cerramos el cursor
cnx.close()     # Cerramos la conexión

print("Conexión cerrada.") #Nos aseguramos que la conexión esté cerrada.


Datos insertados correctamente.
Conexión cerrada.
